# Ajuste nos dados para consulta Big Query
Estou mesclando as colunas repetidas para poder fazer a consulta de data e definir corretamente os compradores de cada lançamento, para então poder seguir com as análises por lançamento.
Além disso houve um problema na formatação:
*"campos de texto com aspas, vírgulas ou quebras de linha não estão sendo corretamente escapados no CSV. A presença de aspas ("), vírgulas (,) ou quebras de linha dentro de campos de texto causa problemas na interpretação do CSV se não forem adequadamente tratados."*
O código abaixo trata ambos os problemas.

In [ ]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime

# PARTE 1: Consolidação de colunas duplicadas
print("Iniciando consolidação de colunas duplicadas...")

# Usar o dataframe existente
df = merged_data.copy()

print(f"DataFrame original: {df.shape[0]} linhas, {df.shape[1]} colunas")

# 1. Consolidar colunas de Qualidade Nome
print("\n--- Consolidando colunas de Qualidade Nome ---")

# Identificar todas as variantes de Qualidade Nome
nome_columns = [
    col for col in df.columns 
    if col.lower() in ['qualidade_nome', 'qualidade (nome)', 'qualidade (nome)', 'qualidade_nome']
    or col.upper() in ['QUALIDADE_NOME', 'QUALIDADE (NOME)', 'QUALIDADE (NOME)', 'QUALIDADE_NOME']
]

# Mostrar colunas encontradas
print(f"Variantes de Qualidade Nome encontradas: {nome_columns}")

# Criar uma única coluna consolidada
if nome_columns:
    # Criar nova coluna
    df['qualidade_nome_consolidada'] = np.nan
    
    # Preencher com valores não nulos de cada variante
    for col in nome_columns:
        mask = df['qualidade_nome_consolidada'].isna() & df[col].notna()
        df.loc[mask, 'qualidade_nome_consolidada'] = df.loc[mask, col]
        
        # Contar valores adicionados desta coluna
        added = mask.sum()
        print(f"- Adicionados {added} valores de '{col}'")
    
    # Estatísticas da consolidação
    total_filled = df['qualidade_nome_consolidada'].notna().sum()
    original_max = max([df[col].notna().sum() for col in nome_columns]) if nome_columns else 0
    
    print(f"- Total na coluna consolidada: {total_filled} valores")
    print(f"- Melhor coluna original tinha: {original_max} valores")
    print(f"- Ganho de {total_filled - original_max} valores ({((total_filled - original_max) / len(df) * 100):.2f}%)")
    
    # Remover colunas originais
    df = df.drop(columns=nome_columns)
    print(f"- Removidas {len(nome_columns)} colunas originais")
else:
    print("Nenhuma coluna de Qualidade Nome encontrada")

# 2. Consolidar colunas de Qualidade Número
print("\n--- Consolidando colunas de Qualidade Número ---")

# Identificar todas as variantes de Qualidade Número
numero_columns = [
    col for col in df.columns 
    if col.lower() in ['qualidade_numero', 'qualidade (número)', 'qualidade (numero)', 'qualidade_numero']
    or col.upper() in ['QUALIDADE_NUMERO', 'QUALIDADE (NÚMERO)', 'QUALIDADE (NUMERO)', 'QUALIDADE_NUMERO']
]

# Mostrar colunas encontradas
print(f"Variantes de Qualidade Número encontradas: {numero_columns}")

# Criar uma única coluna consolidada
if numero_columns:
    # Criar nova coluna
    df['qualidade_numero_consolidada'] = np.nan
    
    # Preencher com valores não nulos de cada variante
    for col in numero_columns:
        mask = df['qualidade_numero_consolidada'].isna() & df[col].notna()
        df.loc[mask, 'qualidade_numero_consolidada'] = df.loc[mask, col]
        
        # Contar valores adicionados desta coluna
        added = mask.sum()
        print(f"- Adicionados {added} valores de '{col}'")
    
    # Estatísticas da consolidação
    total_filled = df['qualidade_numero_consolidada'].notna().sum()
    original_max = max([df[col].notna().sum() for col in numero_columns]) if numero_columns else 0
    
    print(f"- Total na coluna consolidada: {total_filled} valores")
    print(f"- Melhor coluna original tinha: {original_max} valores")
    print(f"- Ganho de {total_filled - original_max} valores ({((total_filled - original_max) / len(df) * 100):.2f}%)")
    
    # Remover colunas originais
    df = df.drop(columns=numero_columns)
    print(f"- Removidas {len(numero_columns)} colunas originais")
else:
    print("Nenhuma coluna de Qualidade Número encontrada")

# 3. Consolidar colunas UTM Campaign
print("\n--- Consolidando colunas UTM Campaign ---")

# Identificar todas as variantes de UTM Campaign
utm_campaign_columns = [
    col for col in df.columns 
    if 'utm' in col.lower() and ('campaign' in col.lower() or 'campaing' in col.lower())
]

# Mostrar colunas encontradas
print(f"Variantes de UTM Campaign encontradas: {utm_campaign_columns}")

# Criar uma única coluna consolidada (utm_campaign)
if utm_campaign_columns and len(utm_campaign_columns) > 1:
    # Criar nova coluna
    df['utm_campaign_consolidada'] = np.nan
    
    # Preencher com valores não nulos de cada variante
    for col in utm_campaign_columns:
        mask = df['utm_campaign_consolidada'].isna() & df[col].notna()
        df.loc[mask, 'utm_campaign_consolidada'] = df.loc[mask, col]
        
        # Contar valores adicionados desta coluna
        added = mask.sum()
        print(f"- Adicionados {added} valores de '{col}'")
    
    # Estatísticas da consolidação
    total_filled = df['utm_campaign_consolidada'].notna().sum()
    original_max = max([df[col].notna().sum() for col in utm_campaign_columns]) if utm_campaign_columns else 0
    
    print(f"- Total na coluna consolidada: {total_filled} valores")
    print(f"- Melhor coluna original tinha: {original_max} valores")
    print(f"- Ganho de {total_filled - original_max} valores ({((total_filled - original_max) / len(df) * 100):.2f}%)")
    
    # Remover colunas originais
    df = df.drop(columns=utm_campaign_columns)
    print(f"- Removidas {len(utm_campaign_columns)} colunas originais")
else:
    print("Nenhuma coluna duplicada de UTM Campaign encontrada")

# PARTE 2: Correção do formato CSV para BigQuery
print("\n=== CORRIGINDO FORMATO CSV PARA BIGQUERY ===")

# 1. Corrigir problemas com aspas e caracteres especiais nos campos de texto
print("Corrigindo problemas de formatação em campos de texto...")

# Identificar colunas de texto
texto_columns = df.select_dtypes(include=['object']).columns.tolist()
print(f"Verificando {len(texto_columns)} colunas de texto")

# Processar cada coluna de texto para remover ou escapar caracteres problemáticos
for col in texto_columns:
    if col in df.columns:  # Verificar se a coluna existe (pode ter sido removida)
        # Substituir aspas duplas por aspas simples
        df[col] = df[col].apply(lambda x: str(x).replace('"', "'") if isinstance(x, str) else x)
        
        # Remover quebras de linha e caracteres de controle que podem afetar o CSV
        df[col] = df[col].apply(lambda x: str(x).replace('\n', ' ').replace('\r', ' ').replace('\t', ' ') 
                               if isinstance(x, str) else x)

print("Correções aplicadas aos campos de texto")

# 2. Tratar valores nulos (opcional - conforme necessidade)
# df = df.fillna('')  # Substituir NaN por strings vazias

# 3. Criar pasta para saída
output_dir = 'bq_export'
os.makedirs(output_dir, exist_ok=True)

# 4. Exportar para CSV com formatação otimizada para BigQuery
print("\nExportando para CSV com formatação otimizada para BigQuery...")

# Gerar nome do arquivo com timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f"{output_dir}/data_consolidada_corrigida_{timestamp}.csv"

# Exportar para CSV com configurações adequadas para BigQuery
df.to_csv(output_file, 
          quoting=csv.QUOTE_NONNUMERIC,  # Coloca aspas em todos os campos não numéricos
          escapechar='\\',               # Usa \ para escapar caracteres especiais
          doublequote=True,              # Duplica aspas internas em vez de escapá-las
          index=False)                   # Não inclui índice

# 5. Exportar também uma versão intermediária para validar
temp_file = f"{output_dir}/data_consolidada_sem_correcao_{timestamp}.csv"
df.to_csv(temp_file, index=False)  # Versão sem as configurações especiais

print(f"Arquivo CSV corrigido exportado para: {output_file}")
print(f"Arquivo CSV padrão (para comparação) exportado para: {temp_file}")

# Resumo final
print("\n=== RESUMO DO PROCESSAMENTO ===")
print(f"DataFrame original: {merged_data.shape[1]} colunas")
print(f"DataFrame consolidado e corrigido: {df.shape[1]} colunas")
print(f"Redução: {merged_data.shape[1] - df.shape[1]} colunas ({((merged_data.shape[1] - df.shape[1])/merged_data.shape[1]*100):.2f}%)")

# Verificar se houve alguma perda de dados
original_non_null = merged_data.count().sum()
new_non_null = df.count().sum()
print(f"Valores não nulos antes: {original_non_null}")
print(f"Valores não nulos depois: {new_non_null}")
print(f"Diferença: {new_non_null - original_non_null}")

print("\nProcessamento concluído com sucesso! Tente importar o arquivo corrigido para o BigQuery.")